## Import pro clustering

In [26]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
data = pd.read_csv("diabetes_prediction_dataset.csv")
data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


In [27]:
X = data.drop('diabetes', axis=1)
y = data['diabetes']

value_counts = X[X['smoking_history'] != 'No Info']['smoking_history'].value_counts()
total_count = value_counts.sum()
distribution = (X['smoking_history'].value_counts() / total_count).drop('No Info')
random= np.random.choice(distribution.index,X[X['smoking_history'] == 'No Info'].shape[0] ,p=distribution.values)

X['smoking_history'].loc[X['smoking_history'] == 'No Info'] = random


X = X.drop(X[X['gender'] == 'Other'].index)
X = pd.get_dummies(X, columns=['gender'])
X = pd.get_dummies(X, columns=['smoking_history'])

C:\Users\cervi\AppData\Local\Temp\ipykernel_15148\640370833.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  X['smoking_history'].loc[X['smoking_history'] == 'No Info'] = random
C:\Users\cervi\AppData\Local\Temp\ipykernel_15148\640370833.

In [28]:
kmeans = KMeans(n_clusters=5, random_state=42,n_init=10)
kmeans.fit(X=X)

KMeans(n_clusters=5, n_init=10, random_state=42)

In [29]:
X

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,gender_Female,gender_Male,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current
0,80.0,0,1,25.19,6.6,140,True,False,False,False,False,True,False
1,54.0,0,0,27.32,6.6,80,True,False,False,False,False,False,True
2,28.0,0,0,27.32,5.7,158,False,True,False,False,False,True,False
3,36.0,0,0,23.45,5.0,155,True,False,True,False,False,False,False
4,76.0,1,1,20.14,4.8,155,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,80.0,0,0,27.32,6.2,90,True,False,False,False,False,True,False
99996,2.0,0,0,17.37,6.5,100,True,False,False,False,False,True,False
99997,66.0,0,0,27.83,5.7,155,False,True,False,False,True,False,False
99998,24.0,0,0,35.42,4.0,100,True,False,False,False,False,True,False


In [35]:
labels = kmeans.labels_
centers = kmeans.cluster_centers_
inertia = kmeans.inertia_



plt.scatter(X['age'], X['hypertension'],X['heart_disease'],X['bmi'],X['HbA1c_level'],X['blood_glucose_level'],X['gender_Female'],X['gender_Male'],X['smoking_history_current'],X['smoking_history_ever'],X['smoking_history_former'],X['smoking_history_never'],X['smoking_history_not current'], c=labels, s=50, cmap='viridis')


#add centers to the plt
plt.scatter(centers[:,0], centers[:,1], marker='x', s=150, c='r', label='centroids')
plt.legend()
plt.show()

TypeError: scatter() got multiple values for argument 'c'